In [1]:
#Notebook Created for Checkpoint 04
#Author - Amit Adate

In [2]:
import numpy as np

In [3]:
import tensorflow as tf

In [4]:
import sklearn

In [5]:
import os
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

In [6]:
corpusdirectory = '/Users/amitadate/Documents/GitHub/MSAI-339-Intro-To-DataScience/Checkpoint-05'
if not os.path.isdir(corpusdir):
    os.mkdir(corpusdir)
    
allFiles = glob.glob(corpusdirectory + "*.txt")

frame = pd.DataFrame(columns=['FileName','Text'])
list_ = []
for file_ in allFiles:
   temp = open(file_,'r')
   corpus_raw = temp.read()
   

In [ ]:
#convert to lower case
corpus_raw = corpus_raw.lower()

words = []

# because we don't want to treat . as a word
for word in corpus_raw.split():
    if word != '.':
        words.append(word)

# remove all duplicate words
words = set(words)

word2int = {}
int2word = {}

# gives the total number of unique words
vocab_size = len(words)


for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word
    






In [ ]:
# we want to add our sentences as a list of words

# raw sentences is a list of sentences.
raw_sentences = corpus_raw.split('.')
sentences = []


for sentence in raw_sentences:
    sentences.append(sentence.split())

In [ ]:
data = []

WINDOW_SIZE = 2

for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

In [ ]:
# function to convert numbers to one hot vectors
def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp

# input word
x_train = [] 

# output word
y_train = []

for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))
    
# convert them to numpy arrays
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

In [ ]:
# we narrowed down to 5 after some tuning and testing
EMBEDDING_DIM = 5 

W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))

#bias
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM])) 

hidden_representation = tf.add(tf.matmul(x,W1), b1)

In [ ]:
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))

b2 = tf.Variable(tf.random_normal([vocab_size]))

# softmax layer
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

In [ ]:
#training
sess = tf.Session()
init = tf.global_variables_initializer()

#running the session
sess.run(init) 

# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))

# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 10000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))

In [ ]:
print(sess.run(W1))
print('----------')
print(sess.run(b1))
print('----------')

In [ ]:
vectors = sess.run(W1 + b1)

# if you work it out, you will see that it has the same effect as running the node hidden representation

In [ ]:
print(vectors)

In [ ]:
print(vectors[ word2int['queen'] ])


In [ ]:
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))

def find_closest(word_index, vectors):
    min_dist = 10000 # to act like positive infinity
    min_index = -1
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
    return min_index

In [ ]:
print(int2word[find_closest(word2int['king'], vectors)])
print(int2word[find_closest(word2int['queen'], vectors)])
print(int2word[find_closest(word2int['royal'], vectors)])

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
model = TSNE(n_components=2, random_state=0)

In [ ]:
#np.set_printoptions(suppress=True)


In [ ]:
#vectors = model.fit_transform(vectors)